In [1]:
import sys
sys.path.append('..')
from CompoundPoisson2 import CompoundPoisson2
import time
import numpy as np
import scipy.stats as st
from SAABS import SA
import pandas as pd
import numpy.linalg as LA
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

In [2]:

class SALoss2(SA):
    def __init__(self, X, c, gamma, K, t, init, epsilon, alpha):
        self.__alpha = alpha
        super(SALoss2, self).__init__(X, c, gamma, K, t, init, epsilon)
    
    @property
    def beta(self):
        return self.__beta
    
    @property
    def alpha(self):
        return self.__alpha
    
    def l(self, m):
        m = np.array(m)
        pos_part = np.maximum(0, m)
        sum_term = np.sum(m)
        square_term = np.sum(np.power(pos_part, 2))
        cross_term = 0
        for i in range(self.dim):
            for j in range(i + 1, self.dim):
                cross_term += pos_part[i] * pos_part[j]
        return sum_term + 0.5 * square_term + self.alpha * cross_term
        
    def grad(self, m):
        m = np.array(m)
        pos_part = np.maximum(0, m)
        sum_pos_part = np.sum(pos_part)
        #this is the vector of sum_{j != i} x_j^+
        pos_part_minus_xi = sum_pos_part - pos_part
        indic = np.sign(pos_part)
        cross_term = np.multiply(indic, pos_part_minus_xi)
        return np.ones(self.dim) + pos_part + self.alpha * cross_term

In [ ]:
#code for compound distribution
#N is the number of steps
#the following code is for construction of compound poisson distribution
d = 10
N = 100000
T = 1
lam = np.random.randint(1, 3, d)
lam_exp = np.random.uniform(0.2, 1.2, d)
mu = [1 for _ in range(d)]
sigma = [1 for _ in range(d)]

R = np.random.randn(d, d)
cov = np.dot(R, np.transpose(R))

rho_gauss = np.ones((d, d))

for i in range(d):
    for j in range(i + 1, d):
        rho_gauss[i, j] = cov[i, j] / np.sqrt(cov[i, i] * cov[j, j])
        rho_gauss[j, i] = rho_gauss[i, j]
C = CompoundPoisson2(T, lam, rho_gauss, N, mu, sigma)
rho_pois = C.get_rho_pois()
#jump size with gaussian distribution 
cov_matrix_X_gauss_jump = C.cov_final_X_gauss()
distr = C.compound_poisson2_gauss()

In [ ]:
#alpha is the systemic weight
alpha = 1.
#c, gamma of the SA
c, gamma  = 6., 0.7
#K the projection compact
K = [(-1, 20) for _ in range(d)] + [(0, 10)]
#t for the window of averaging
t = 10
#epsilon for the calculation of the approximation of the Jacobian matrix A
epsilon = 10 ** (-6)
#init is Z_0 of the SA
init = np.zeros(d + 1)
start_time = time.time()
loss = SALoss2(distr, c, gamma, K, t, init, epsilon, alpha)
#Z0 with gamma = 0.8
Z0 = loss.setRM()
print("Computation Time RM Algorithm: %s seconds ---" % (time.time() - start_time))

In [ ]:
#PR averaging
#estimation of Sigma* and the jacobian matrix A
loss.setEst()
zBar, CI = loss.getPR()
print(zBar)